In [3]:
#%matplotlib inline
import OifFile as of
import numpy as np
import matplotlib.pyplot as plt
import os
from __future__ import division
import sys
sys.path.insert(1,'T:\\James_Rowland\\alpha6_Cre_ERT')

def file_loader():
    '''Loads Tiff images and returns them as arrays in a list''' 
    #get list of all files in dir
    dirList = os.listdir('T:\\James_Rowland\\alpha6_Cre-ERT\\151005')   
    
    dirList.sort(key=lambda x: os.stat(os.path.join('T:\\James_Rowland\\alpha6_Cre-ERT\\151005', x)).st_mtime)
    
    return dirList
    
def oif_selector():  

    oif_files = []
    
    file_list = file_loader()
    
    for files in file_list:
        if files.endswith((".oif")):
            oif_files.append(files)
            
    global lessrepeats
    
    lessrepeats = oif_files[::3]
    
    return oif_files

def array_list_creator():
        
    oif_file_list = oif_selector()

    
    #load the files corresponding to the lists into lists of arrays
    oif_array_list = []
    
    for fileName in oif_file_list:
        oif_array_list.append(of.OifFile(fileName).asarray())
    
    #finally return the list of arrays
    return oif_array_list

def get_percentage_expressing():
    
    arraylist = array_list_creator()
    
    y = [] 
    
    for item in arraylist:
       
        DAPI = item[:,0,:,:]
        TdTomato = item[:,1,:,:]
        #indexes to split the two channels

        D_reshaped = DAPI.reshape(DAPI.shape[0], -1)
        Td_reshaped = TdTomato.reshape(TdTomato.shape[0], -1)
        # reshapes the array to 39L, 262144L - merging 512 x 512 into a single dimension

        Dav = np.average(D_reshaped, axis=1)
        Tdav = np.average(Td_reshaped, axis=1)
        # average pixel intensity of each slice 

        Dstd = np.std(D_reshaped, axis = 1)
        Tdstd = np.std(Td_reshaped, axis = 1)
        # standard deviation of pixel intensities of each slice 

        DAPI_thresh = Dav + Dstd
        #this threshold will only hold true at bright nuclei

        DAPIbroad = DAPI_thresh[:, np.newaxis, np.newaxis]
        # generates new dimensions to the DAPI threshold, allowing it to be used in operations

        import numpy.ma as ma

        masked_DAPI = ma.masked_greater(DAPI, DAPIbroad)
        #returns true if the value definitely lies in a nucleus

        Td_background = masked_DAPI.mask * TdTomato
        Td_back_reshape = Td_background.reshape(TdTomato.shape[0], -1)
        back_av = np.average(Td_back_reshape, axis = 1, weights=Td_back_reshape.astype(bool))
        #generates the average intensity of the background at each slice - weight excludes 0 values 

        Td_thresh = back_av + Tdstd
        Tdbroad = Td_thresh[:, np.newaxis, np.newaxis]
        masked_Td = ma.masked_greater(TdTomato, Tdbroad)
        # generates a masked array returning true for pixels over threshold

        number_pixels_above_threshold = np.sum(masked_Td.mask)
        number_background_pixels = np.sum(masked_DAPI.mask)

        percentage_expressing = (number_pixels_above_threshold / ((DAPI.shape[0]*DAPI.shape[1]*DAPI.shape[2])-number_background_pixels))*100

        y.append(percentage_expressing)
        
    results_array= np.asarray(y)
    shaped = results_array.reshape(-1 ,3)
    average = shaped.mean(axis=1)
    #averages the list of results in groups of three
    
    #print average
    
    return average

def data_processor():
    
    
    results = get_percentage_expressing()
    
    why = results[:]
    #this piece of code seems pointless but it seems to debug the issue of replicating the array 

    reslist = why.tolist()
    
    label = []
    
    for files in lessrepeats:
        label.append(files[0:3])
    #simplifies list of animal names down to founder
    
    label.sort()
        
    lesslabel = []
    
    for i in label:
        if i not in lesslabel:
            lesslabel.append(i)
    #removes different animals, leaving just the founder
 
        
    numkey = []
        
    for data in lessrepeats:
        numkey.append(data[0:1])
    
    intkey = []
    
    for nums in numkey:
        intkey.append(int(nums))
    #these three lines of code turn the animal name into a plottable number
        
    xaxis = [] 
    
    for i in intkey:
        if i not in xaxis:
            xaxis.append(i)
    
    xaxis.sort()
    #creates a list that will correctly position the x axis labels
    
    ymax = max(reslist) + 3
    plt.scatter(intkey, reslist)
    plt.xticks(xaxis, lesslabel)
    x1,x2,y1,y2 = plt.axis()
    plt.axis((x1,x2,0,ymax))
    #sets the y axis range
    plt.ylabel('Percentage of Pixels Above Threshold')
    plt.xlabel('Founder Mouse')
    plt.show()
    
    
data_processor()
        

